In [1]:
import numpy as np
import pandas as pd
from equilibrator_api import ComponentContribution, Q_
CC = ComponentContribution()
from equilibrator_assets.generate_compound import create_compound, get_or_create_compound
from ast import literal_eval

In [2]:
def compound(filepath, name):
    df = pd.read_csv(filepath, sep='\t') 
    indexes = []
    compounds = []
    Reagents = []
    Products = []
    rules = []
    for i in range(len(df['Index'])):
        indexes.append(df['Index'][i])
        rules.append(df['Rule'][i])
        reagents = literal_eval(df['Reagents'][i])
        Reagents.append(reagents)
        products = literal_eval(df['Products'][i])
        Products.append(products)
        for j in range(len(reagents)):
            if reagents[j] not in compounds:
                compounds.append(reagents[j])
        for k in range(len(products)):
            if products[k] not in compounds:
                compounds.append(products[k])
    
    print('Checkpoint 1')
    
    compound_list = get_or_create_compound(CC.ccache, compounds, mol_format="smiles")
    
    print('Checkpoint 2')
    
    return(compound_list)

In [3]:
def ThermoGen(filepath, name, comlist):
    df = pd.read_csv(filepath, sep='\t') 
    indexes = []
    compounds = []
    Reagents = []
    Products = []
    rules = []
    for i in range(len(df['Index'])):
        indexes.append(df['Index'][i])
        rules.append(df['Rule'][i])
        reagents = literal_eval(df['Reagents'][i])
        Reagents.append(reagents)
        products = literal_eval(df['Products'][i])
        Products.append(products)
        for j in range(len(reagents)):
            if reagents[j] not in compounds:
                compounds.append(reagents[j])
        for k in range(len(products)):
            if products[k] not in compounds:
                compounds.append(products[k])
    
    print('Checkpoint 3')
    
    compound_list = comlist
    
    print('Checkpoint 4')
    
    mus = []
    sigma_vecs = []
    for c in compound_list:
        mu = (CC.predictor.preprocess.get_compound_prediction(c))[0]
        sigma_vec = (CC.predictor.preprocess.get_compound_prediction(c))[1]
        mus.append(mu)
        sigma_vecs.append(sigma_vec)
    
    print('Checkpoint 5')
    
    EnergyChanges = []
    for i in range(len(df['Index'])):
        print(i)
        dummy_mus = []
        dummy_sigma_vecs = []
        dummy_compounds = []
        dummy_coefficients = []
        reagents = literal_eval(df['Reagents'][i])
        products = literal_eval(df['Products'][i])
        for j in range(len(reagents)):
            dummy_compounds.append(reagents[j])
            dummy_coefficients.append(-1)
        for k in range(len(products)):
            dummy_compounds.append(products[k])
            dummy_coefficients.append(1)
        for m in range(len(dummy_compounds)):
            dummy_mus.append(mus[compounds.index(dummy_compounds[m])])
            dummy_sigma_vecs.append(sigma_vecs[compounds.index(dummy_compounds[m])])
        
        S = np.zeros(len(dummy_compounds))
        for n in range(len(dummy_coefficients)):
            S[n] = dummy_coefficients[n]
        dummy_mus = Q_(dummy_mus, "kJ/mol")
        dummy_sigma_vecs = Q_(dummy_sigma_vecs, "kJ/mol")
        standard_dgs = S.T @ dummy_mus
        U = S.T @ dummy_sigma_vecs
        EnergyChanges.append(standard_dgs._magnitude.round(2))
        
    outputdata = {'Index':indexes, 'Reagents':Reagents, 'Products':Products, 'Rule':rules, 'Energy Change':EnergyChanges}
    outputdf = pd.DataFrame(outputdata)
    outputdf.to_csv(f'{name}RelsWithThermo.tsv', header=None, index=None, sep='\t', mode='a')
    return(outputdf)

In [4]:
c = compound('PyruvicAcidG3processedrels.tsv', 'PyruvicAcid3')

Checkpoint 1


Checkpoint 2


In [8]:
ThermoGen('PyruvicAcidG3processedrels.tsv', 'PyruvicAcid3', c)

Checkpoint 1again
Checkpoint 2again
Checkpoint 3
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90


,Index,Reagents,Products,Rule,Energy Change
0,2491_0,"[CO, C(C1(C=C(C(O1)=O)C)O)(O)=O]",[C1(C(C)C(CO)C(C(O)=O)(O)O1)=O],"Michael Addition 0,2,",1178.48
1,2493_0,"[CO, C(C1(CC(C(O1)=O)=C)O)(O)=O]",[C1(C(CC(C(O)=O)(O)O1)CCO)=O],"Michael Addition 0,2,",994.29
2,2495_0,"[CO, C1(C(C)=CC(C(O1)=O)=O)=O]",[C1(C(C)C(C(C(O1)=O)=O)CO)=O],"Michael Addition 0,2,",567.51
3,2497_0,"[CO, C1(C(C)=CC(C(O1)=O)=O)=O]",[C1(C(C)(CO)CC(C(O1)=O)=O)=O],"Michael Addition 0,2,",376.93
4,2499_0,"[CO, C=C1C(OC(C(C1)=O)=O)=O]",[C1(C(CCO)CC(C(O1)=O)=O)=O],"Michael Addition 0,2,",472.56
...,...,...,...,...,...
86,2663_0,"[C(CO)O, C(CC(C(O)=O)=C)=O]",[C(C(CC(CO)O)CC=O)(O)=O],"Michael Addition 0,2,",4.75
87,2665_0,"[C(CO)O, C(C=C(C(O)=O)C)=O]",[C(C(C)(C(CO)O)CC=O)(O)=O],"Michael Addition 0,2,",775.68
88,2667_0,"[C(CO)O, C(C=C(C(O)=O)C)=O]",[C(C(C)C(C=O)C(CO)O)(O)=O],"Michael Addition 0,2,",775.68
89,2668_0,"[C(C(C)=O)(O)=O, C(C)=CC=O]",[C(CC(C)CC(C(O)=O)=O)=O],"Michael Addition 0,2,",602.89
